# High Rotation Rotator/CCW Integration test

This notebook relates to testcase 

https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1594

This notebook performs a high rotation stress scenario integration test between the Rotator and the Camera Cable Wrap with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import MTPtg

from astropy.utils import iers
iers.conf.auto_download = False
from datetime import datetime 

In [ ]:
test_message = "High Rotation Rotator_CCW Integration Test"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "MTRotator")
mtm = salobj.Remote(d, "MTMount")
mtptg = salobj.Remote(d, "MTPtg")

In [ ]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task, 
                     script.start_task, 
                     mtm.start_task)

In [ ]:
now = datetime.now()
print(test_message, now)

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mtm, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.OFFLINE)

In [ ]:
CCWstate = await mtm.evt_cameraCableWrapFollowing.aget(timeout=10.)
print("CCWstate:",CCWstate.enabled)
await mtm.cmd_enableCameraCableWrapFollowing.set_start(timeout=10.)
CCWstate = await mtm.evt_cameraCableWrapFollowing.aget(timeout=10.)
print("CCWstate:",CCWstate.enabled)

# Track Rotator to +90deg

In [ ]:
print("Test Start")

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Track Rotator to +90deg - {now} UTC")
await asyncio.sleep(0.1)

print("Track to +90.0 deg for starting position")

telemetry = rot.tel_rotation.get()
cpos = telemetry.actualPosition
print(cpos)

vel = 0.068
dt = (90 - cpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    pos = cpos + i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    print("pos:", pos)
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)
now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message} - Track Rotator to +90deg - {now} UTC")

# Perform Track from +90deg to -90deg

In [ ]:
print("Test Start")

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Perform Track from +90deg to -90deg - {now} UTC")
await asyncio.sleep(0.1)

print("Track from +90 deg to -90.0 deg at max velocity")

telemetry = rot.tel_rotation.get()
actualpos = telemetry.actualPosition
print("actualpos", actualpos )

vel = 0.068
dt = (90 + actualpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)
print("steps:", steps)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    pos = cpos - i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    print("pos:", pos)
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)

# Move to Zero

In [ ]:
now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to Zero - {now} UTC")
await asyncio.sleep(0.1)

print("Move to 0.0 deg")

rot.evt_inPosition.flush()

await rot.cmd_move.set_start(position=0.0)

inPOS = await rot.evt_inPosition.aget()

print(inPOS.inPosition)

print("Test complete")

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message} - Move to Zero - {now} UTC")

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_move.set_start(position=19.04,timeout=30.)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearError.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring Mount/CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)